In [ ]:
%load_ext autoreload
%autoreload 2

import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import formulas
import pandas as pd
import random
import dataframes

plotly.offline.init_notebook_mode(connected=True)
# pd.set_option("display.max_rows", None, "display.max_columns", None)

# Inputs
Assume fixed rate mortgage

In [ ]:
# Loan
DOWN_PAYMENT = 200000
PRINCIPAL = 900000 # This is the value of house. Principal is probably a bad name for this.
LOAN_APR = 0.077
DURATION = 30 # number of years

# Additional Costs
# Home insurance 1500 per year
# HOA 150 per month
ADDITIONAL_MONTHLY_COSTS = 1500/12 + 150

# Tax
TAX_APPRAISAL_VAL = PRINCIPAL
TAX_APR = 0.02
TAX_APPRAISAL_VAL_GROWTH_RATE = 0.0

# Income
MONTHLY_INCOME = 200000/12 # Gross monthly income because debt-to-income ratio is calculated from gross

# Randomized Interest Rates
INT_RANGE_LOW = 0.02
INT_RANGE_HI = 0.11
INT_CHANGE_PERIOD_YEARS = 3 # randomize each number of years

# Don't touch
PAYMENTS_N = 12 # monthly payments. CAUTION - this should probably never be changed because I'm sure I've hardcoded 12 elsewhere
loan_value  = PRINCIPAL - DOWN_PAYMENT

# Property Tax

<span style="color:red">
</span>

In [ ]:
tax_df = dataframes.property_tax_amortization(TAX_APPRAISAL_VAL, TAX_APR, DURATION, TAX_APPRAISAL_VAL_GROWTH_RATE)
# display(tax_df.round(0))

# Mortgage Amortization

In [ ]:
ma_df = dataframes.mortgage_amortization(loan_value, LOAN_APR, PAYMENTS_N, DURATION, DOWN_PAYMENT)
display(ma_df.round(0))

fig = px.bar(ma_df, x=ma_df.index, y=['interest paid', 'principal paid'])
fig.add_scatter(x=ma_df.index, y=ma_df['mortgage balance'], name='mortgage balance')
fig.add_scatter(x=ma_df.index, y=ma_df['equity'], name='equity')
fig.show()

# Payment Amortization

In [ ]:
as_df = dataframes.amortization_summary(ma_df, tax_df, monthly_income=MONTHLY_INCOME, monthly_additional_costs=ADDITIONAL_MONTHLY_COSTS)
display(as_df.round(0))

#Sums
fig = px.bar(as_df, x=as_df.index, y=['principal paid', 'interest paid', 'tax paid'], title='Amounts Paid')
fig.add_scatter(x=as_df.index, y=as_df['mortgage balance'], name='mortgage balance', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['equity'], name='equity', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['payments'], name='payments', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['appraisal value'], name='appraisal value', mode='lines')
fig.show()


# Monthly Payments

In [ ]:
fig = px.bar(as_df, x=as_df.index, y=['mortgage payment principal', 'mortgage payment interest', 'monthly tax payment'], title='Monthly Payments')
fig.add_scatter(x=as_df.index, y=as_df['monthly payment'], name='monthly payment', mode='lines')
fig.show()

# Debt to Income Ratio

In [ ]:
fig = px.scatter(as_df, x=as_df.index, y=['debt to income ratio'], title='Debt to Income Ratio')
fig.update_yaxes(range=[0, 100])
fig.show()

# Randomized Interest Rate
Randomize the interest rate every few years. Re-finance the loan into this new rate regardless of whether it's higher or lower.

In [ ]:
ri_df, periodic_interest_rates = dataframes.randomized_interest(LOAN_APR, INT_RANGE_LOW, INT_RANGE_HI, INT_CHANGE_PERIOD_YEARS, DURATION * PAYMENTS_N)
fig = px.scatter(ri_df, x=ri_df.index, y=['interest rate'], title='Randomized Interest Rate')
fig.show()
#print(periodic_interest_rates)

# comment this out to not debug
# periodic_interest_rates = [0.077, 0.07309579158741138, 0.046674635865706436, 0.10835080679897587, 0.0968527302624414, 0.08129664568334291, 0.10207750869120687, 0.08102664288489458, 0.08718379322627176, 0.04845177074687092]
# periodic_interest_rates = [LOAN_APR for i in range(DURATION // INT_CHANGE_PERIOD_YEARS)]

ma_df = dataframes.refi_mortgage_amortization(PRINCIPAL, PAYMENTS_N, DURATION, DOWN_PAYMENT, INT_CHANGE_PERIOD_YEARS, periodic_interest_rates)

# debug remove me
for i in range(len(periodic_interest_rates)):
  start = i*PAYMENTS_N*INT_CHANGE_PERIOD_YEARS
  end = start + 2
  display(ma_df[start:end].round(0))
display(ma_df[360:361].round(0))

# display(ma_df.round(0))

fig = px.bar(ma_df, x=ma_df.index, y=['interest paid', 'principal paid'])
fig.add_scatter(x=ma_df.index, y=ma_df['mortgage balance'], name='mortgage balance')
fig.add_scatter(x=ma_df.index, y=ma_df['equity'], name='equity')
fig.show()